# EXP4268-PY PUD break Analysis to estimate manipulated breaks

Generate PUD stop report through **[this link](https://opms.dhl.com/opms/report/pudStop/index.jsp?guiStatusGZ=z3c-f6tRqldQoncW869ipmoyrlZKp6ZMfnpIcjlUoSwLtOUJcz1OfAp5cQ5klB1VeioXjQhlj3x9gQQTgCII012345652784565279abcde53fcga6hie9jkele9m6n2ceope9qfnh78r2aces55k8ttuuuva273hc77rjicvict7rscf6tari23tr2acwxvyvye9qfnh78ifes55k8ttuuuva273hc77rjicvict7rscf6tari23ti3ffc67wovyvye9qfnh78des55k8ttuuuva273hc77rjicvict7rscf6tari23td3n5czwovyvye9qfnh78123456527es55k8ttuuuva273hc77rjicvict7rscf6tari23t123456527wovyvyeA0d8B3n5czA0d8Cqkzc773jh9oe53fcga6hie9xele9DeopetA0d8Ehi93iekcz3jie9i65cBzjfexyxxwoxwyoe9i65cFjexyxxwoxwope937G6n3ie5z2ce9ariHj75d3qeariCfk5IeA0J9oeiIe9xeK15Lle9DexyxxwoxwyoetA0J9oeiIe9xeKn5Lle9DexyxxwoxwopetA0A0d8Mz9ariN3fch73jheHONPjr653jhe9aribci37kn6IeQeA0oyA0J9oek2igr5zgrjice9xele9DePGe9r6k53jhePEFGREetA0A0A0A01234565278N3fch73jh9oek2ignjr653jhe9xele9Derj2h5zIe9oJeSj2h5zIetA01234565278Cqk6hiT2n539oUeHONPjr653jhe9Ded6n7cetA0r2ac8P6Ij259r2acRier2ace9oQeyeA0r2ac8N3fch73jh79s3icFj56n7e5z2ceA0r2ac8bju7A0if8Pcmcnbcd95j56ne5z63n3h1e9oJeHON9bj25ce9zci2rcie5z2ce97rjkcek2iVjjWXk2i4s3kXk2i45jke9oezj25cg3ietA0xU9xQe5z63n3h1e9oJeHON9N65ce9Dyek2i45jke9zcdN3fch73jhek2iHcz3jie9oek2igiIgi3fetA0A0r2ac8Sjn2fh7A0xU9xQe5z63n3h1e9oJeF3fca6hi93h9T3h25c7e9oe53fcga6hietA0A0A0r2ac8Tc672zc7A0r2ac8EnnN3f7A0if8Tc672zcbcd9oobcdeazc6Wgf3he9oJeVzc6W9f3h25c7etA0A0A0A0A)**. Update the country name and analysis period. 


If the total is not hidden from the report, please hide it.


<img src = "/files/images/Hide_total.JPG" width = 400 />

Generally one month is a fair time for this analysis, otherwise pay close attention to the OPMS maximum row limit while selecting analysis period.It is also possible to extract report in multiple parts i-e splitting the time period and appending all the reports in one excel sheet.


<img src = "/files/images/OPMS_max_row_limit.JPG" width = 400 />

Export the excel report (NOT the detail Excel report) 

<img src = "/files/images/export_excel.JPG" width = 400 />

Open extracted report and delete first three lines to bring the headers in first two rows.

<img src = "/files/images/remove_top3_rows.JPG" width = 400 />

## Save this file in your desired location and insert the location below as "in_put"

In [1]:
import pandas as pd
in_put = 'C:\\Users\\muhzafar\\Downloads\\export_20230207T1250.xlsx'
df = pd.read_excel(in_put)

In [2]:
df = df.reset_index(drop=True)

# Replacing unmerged cells with their corresponding values
df['PUD Route'] = df['PUD Route'].fillna(method='ffill')
df['PUD Date'] = df['PUD Date'].fillna(method='ffill')


In [3]:
# Select only the specified range of columns and rows
df['Estimated number of courier manipulated breaks'] = ((df.iloc[1:, 2:] != 15) & (df.iloc[1:, 2:].notnull())).sum(axis=1)

In [4]:
df = df.reset_index(drop=True)
df['breaks per route per analysis period'] = df.iloc[1:].groupby(df.columns[0])[df.columns[-1]].transform(lambda x: x.sum() if (x.notnull().all() and x.astype(float).dropna().size > 0) else 0)
df = df.reset_index(drop=True)

In [5]:
cols = df.columns.tolist()
df.insert(0, cols[-2], df.pop(cols[-2]))
df.insert(0, cols[-1], df.pop(cols[-1]))
df.sort_values(by=df.columns[0], ascending=False, inplace=True)
df = df[df.ne('Break minutes').all(axis=1)] #remove unnecessary row in header 


## Insert the output location to save final results

In [6]:
out_put = 'C:\\Users\\muhzafar\\Downloads\\EXP4268-PY.xlsx'
df.to_excel(out_put, index = False, sheet_name = 'EXP4268-PY')

In [7]:
import openpyxl

# Load the Excel workbook
wb = openpyxl.load_workbook(out_put)

# Select the active sheet
sheet = wb.active

# Select the cell to add a comment to
cell = sheet.cell(row=1, column=1)

# Add a comment to the cell
cell.comment = openpyxl.comments.Comment('Toqeer: In Descending order, the routes with most no of manipulated breaks', 'Toqeer:')

cell2 = sheet.cell(row=1, column=2)

# Add a comment to the cell
cell2.comment = openpyxl.comments.Comment("Toqeer: Breaks equal to 15min are not conunted and are assumed here as regular breaks", 'Toqeer:')

# Save the changes to the workbook
wb.save(out_put)
